Welcome! This IPython notebook does a sensitivity analysis of the mobility model for the water access research project.

Import Packages

In [1]:
import pandas as pd
import mobility_module as mm
import numpy as np
import plotly
import plotly.express as px
import plotting_tools_water_access
from plotly.subplots import make_subplots
import plotly.graph_objects as go



Import data from csv files

In [2]:
######################
#### Import Data #####
with open("../data/mobility-model-parameters.csv") as csv_file:
    # read the csv file
    allHPV_param_df = pd.read_csv("../data/mobility-model-parameters.csv")


# PULL FROM CSV
# get linear space for variable range
with open("../data/mobility-model-parameters.csv") as csv_file:
    # read the csv file
    sens_df = pd.read_csv("../data/Sensitivity Analysis Variables.csv")


Initiliase Velocity Kgs, define colours for graphing

In [3]:
graph_colours = ["#3D87CB", "#F0B323", "#DC582A", "#674230", "#3A913F", "#75787B"]
full_result_dict = {}
plot_dict = {}
df_large = pd.DataFrame()


### FIlter Data
col = "Name"
filter_value = "Bicycle"
param_df = allHPV_param_df.loc[(allHPV_param_df[col] == filter_value)]

for i in range(0, len(sens_df)):

    # # initialise variables, options, MET, hpv and Velocity Kgs classes, populated with data via 'init' functions
    mo = mm.model_options()
    mv = mm.model_variables()
    met = mm.MET_values(mv)
    hpv = mm.HPV_variables(param_df, mv)
    mr = mm.model_results(hpv, mo)

    # select model (bike...)
    mo.model_selection = 2


    ###### Start big if statement area
    ##### SENSITIVITY
    res = 7

    # set up variables for sensitivity analysis
    plot_min = sens_df.iloc[i]["Plotting Min"]
    plot_max = sens_df.iloc[i]["Plotting Max"]
    minval = sens_df.iloc[i]["Expected Min"]
    maxval = sens_df.iloc[i]["Expected Max"]
    def_val = sens_df.iloc[i]["Default Value"]
    var_string = sens_df.iloc[i]["Short Name"] # change
    var_units = sens_df.iloc[i]["Units"] # change

    # create linear space of weights
    phase_space = np.linspace(
        start=plot_min,  # define linear space
        stop=plot_max,  # define maximum value for linear space
        num=res,  # how many data points?
        endpoint=True,
        retstep=False,
        dtype=None,
    )
    # include the default, max, and min values seperatly so they are easy to see graphically
    # phase_space = np.append(phase_space,hpv.load_limit.flatten()[0])
    phase_space = np.append(phase_space,def_val)
    phase_space = np.append(phase_space,minval)
    phase_space = np.append(phase_space,maxval)

    trip_velocity_mean = []  # create empty list to place variables in to in loop
    unloaded_velocity_mean = []  # create empty list to place variables in to in loop
    loaded_velocity_mean = []  # create empty list to place variables in to in loop
    velocitykgs = []  # create empty list to place variables in to in loop
    water_ration_kms = []  # create empty list to place variables in to in loop

    for var_test in phase_space:
        ## THIS WILL NEED A FAT IF STATEMENT AS WE NEED TO CHANGE TARGET of the assignment AND the value AND there are differening data storgae types (i.e numpy 3 dims, scalar etc)
        if var_string == "Coefficient of Rolling Resistance":
            hpv.Crr = np.array([[[var_test]]]) # change
        elif var_string == "Load Limit":
            hpv.load_limit = np.array([[[var_test]]]) # change
        elif var_string == "Reference Area":
            mv.A = var_test 
        elif var_string == "Drag Coefficient":
            mv.C_d = var_test 
        elif var_string == "Efficiency":
            mv.eta = var_test 
        elif var_string == "Air Density":
            mv.ro = var_test 
        elif var_string == "MET budget":
            met.MET_of_sustainable_excercise = var_test 
        elif var_string == "T_hours":
            mv.t_hours = var_test 
        elif var_string == "HPV Weight":
            hpv.m_HPV_only = np.array([[[var_test]]]) # change
        elif var_string == "Water Ration":
            mv.waterration = var_test 
        elif var_string == "Human Weight":
            mv.m1 = var_test 
        elif var_string == "Human Power Output":
            mv.P_t = var_test 
            


        
        
        # # hpv.load_limit = np.array([[[var_test]]]) # change
        # mv.C_d = var_test # change

        ######## Numerical MODEL ########
        mr.v_load_matrix3d, mr.load_matrix3d = mm.mobility_models.numerical_mobility_model(
            mr, mv, mo, met, hpv
        )
        ####### Organise Velocity Kgs #######
        mr.load_results(hpv, mv, mo)

        maximum_vel_kg_per_slope = np.amax(mr.velocitykgs, axis=2)
        mean_vel_kg_per_slope = np.mean(maximum_vel_kg_per_slope)

        # append to list
        trip_velocity_mean.append(np.mean(mr.v_avg_matrix3d))
        unloaded_velocity_mean.append(np.mean(mr.v_unload_matrix3d))
        loaded_velocity_mean.append(np.mean(mr.v_load_matrix3d))
        velocitykgs.append(mean_vel_kg_per_slope)
        water_ration_kms.append(mean_vel_kg_per_slope/mv.waterration*mr.t_secs/1000) # [kg][m/s] * [s] * 1/[kg] * 1/1000 = [km]. Divide by distance to water, and accomodate for household size to see if needs are met.



    small_marker_size = 10
    med_marker_size = 20
    large_marker_size = 50
    data = {'Velocity Kgs': velocitykgs,'Water Ration Kms': water_ration_kms, 'Mean Trip Velocity': trip_velocity_mean, 'Mean Unloaded Velocity': unloaded_velocity_mean, 'Mean Loaded Velocity': loaded_velocity_mean}
    df_results = pd.DataFrame(data)

    df_results["Adjusted Result"] =  df_results["Velocity Kgs"] - df_results.at[res,'Velocity Kgs']
    df_results["Adjusted Water Ration Kms"] =  df_results["Water Ration Kms"] - df_results.at[res,'Water Ration Kms']
    df_results["Variable"] = phase_space
    df_results["DataType"] = "Sensitivity"
    df_results["MarkerSize"] = small_marker_size
    df_results["Name"] = var_string
    df_results.at[res,'DataType'] = "Default"
    df_results.at[res+1,'DataType'] = "Minimum Expected"
    df_results.at[res+2,'DataType'] = "Maximum Expected"
    df_results.at[res+1,'MarkerSize'] = med_marker_size
    df_results.at[res+2,'MarkerSize'] = med_marker_size
    df_results.at[res,'MarkerSize'] = large_marker_size


    full_result_dict[var_string] = df_results
    df_large = pd.concat([df_large, df_results], sort=False)



In [4]:

# for i in range(0, len(sens_df)):
primary_graph_value = "Water Ration Kms"
max_plot_y = df_large[primary_graph_value].max()*1.10
min_plot_y = df_large[primary_graph_value].min()*0.90


for i in range(0, len(sens_df)):

    fig1 = px.scatter(
        full_result_dict[sens_df.iloc[i]["Short Name"]],
        x="Variable",
        y=primary_graph_value,
        color='DataType',
        color_discrete_sequence=graph_colours,
        size='MarkerSize',
        title=f'Effect of {sens_df.iloc[i]["Short Name"]} on model Velocity Kgs',
        hover_data=[
            "Velocity Kgs",
            'Water Ration Kms',
            'Mean Trip Velocity',
            'Mean Loaded Velocity',
            'Mean Unloaded Velocity',
        ]


    ).update_layout(
        yaxis_title=primary_graph_value,
        xaxis_title=(f'{sens_df.iloc[i]["Short Name"]}  <i>{sens_df.iloc[i]["Units"]}</i>'),
    )
    fig1.update_yaxes(range = [min_plot_y, max_plot_y])

    fig1 = plotting_tools_water_access.format_plotly_graphs(fig1)

    fig1.show()
    plot_dict[var_string]= fig1
    # filename_string = var_string + ".html"
    # plotly.offline.plot(fig1, filename=filename_string)

In [5]:
df_large[primary_graph_value].max()*1.10
df_large[primary_graph_value].min()*0.90

41.86960610228119

In [8]:


summary_df = df_large[(df_large.DataType == "Minimum Expected") | (df_large.DataType == "Maximum Expected") | (df_large.DataType == "Default")]
# summary_df
fig = px.bar(summary_df, y="Name", x="Adjusted Water Ration Kms", color="DataType", title="Sensitivity Of Variables",        #size='MarkerSize',
)
fig2 = px.bar(summary_df, y="Name", x="Water Ration Kms", color="DataType", title="Sensitivty Of Variables",        #size='MarkerSize',)
)
# fig2.update_traces(xaxis="x2")

# fig=make_subplots(
#         specs=[[{"secondary_y": True}]])
# print(fig.layout)    



# subfig = make_subplots(specs=[[{"secondary_y": True}]])
# subfig.add_traces(fig.data + fig2.data)

# subfig.layout.xaxis.title="Time"
# subfig.layout.yaxis.title="Linear Y"
# subfig.layout.yaxis2.title="Log Y"


fig.show()

fig.write_image("../results/fig1.svg")


# fig.update_layout(xaxis2= {'anchor': 'y', 'overlaying': 'x', 'side': 'top'},
#                   yaxis_domain=[0, 0.94]);

# fig.add_trace(
#     go.Bar(x=[1, 2, 3, 4],
#            y=[7, 4, 5, 6],
#            name="bar",
#           ), secondary_y=False)
# fig.add_trace(
#     go.Bar(x=[-2, -1, 0, 1],
#                y=[4, 2, 5, 6],
#                name="scatt1",
#             ), secondary_y=True)
# fig.data[1].update(xaxis='x2')
# fig.update_layout(width=700, height=475)



In [ ]:
# import plotly.graph_objects as go

# fig=make_subplots(
#         specs=[[{"secondary_y": True}]])
# print(fig.layout)    

# fig.update_layout(xaxis2= {'anchor': 'y', 'overlaying': 'x', 'side': 'top'},
#                   yaxis_domain=[0, 0.94]);

# fig.add_trace(
#     go.Bar(x=[1, 2, 3, 4],
#            y=[7, 4, 5, 6],
#            name="bar",
#           ), secondary_y=False)
# fig.add_trace(
#     go.Bar(x=[-2, -1, 0, 1],
#                y=[4, 2, 5, 6],
#                name="scatt1",
#             ), secondary_y=True)
# fig.data[1].update(xaxis='x2')
# fig.update_layout(width=700, height=475)

In [ ]:
summary_df["Name"].to_csv("listvars.csv")